In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window
from datetime import date, timedelta, datetime

#### data

In [0]:
#Get Variables
#recuperer la date d'aujourd'hui
today = date.today()
#recuperer la date d'hier 
yesterday = today - timedelta(days = 1)
#à partier de la date d'hier on recuper l'année ,le mois, et le jour 
year = yesterday.year
month = yesterday.month
day = yesterday.day

In [0]:
#on recupere que les données d'hier pour l'automatisation 
path = f"/mnt/processed-prod/Daily/SpeedRoadType/"+str(year)+'/'+'{:02d}'.format(month)+'/'+'{:02d}'.format(day)+'/'
data= spark.read.option("header",True).option("inferSchema",True).parquet(path)

#### Les fonctions

In [0]:
#fonction pour calculer la distance
def get_distance(longit_a, latit_a, longit_b, latit_b):
  if longit_a is None or latit_a is None or longit_b is None or latit_b is None:
    return 0
  # Transform to radians
  longit_a, latit_a, longit_b, latit_b = map(radians, [longit_a, latit_a, longit_b, latit_b])
  dist_longit = longit_b - longit_a
  dist_latit = latit_b - latit_a
  # Calculate area
  area = sin(dist_latit / 2) ** 2 + cos(latit_a) * cos(latit_b) * sin(dist_longit / 2) ** 2
  # Calculate the central angle
  central_angle = 2 * asin(sqrt(area))
  radius = 6371
  # Calculate Distance in km
  distance = (central_angle * radius)
  return abs(distance)

In [0]:
def ajouter_distance(data):
  data_lag=data
  #calculer la fonction lag pour calculer la distance de chaque trajet
  w = Window().partitionBy("rideid").orderBy(col("deviceid").asc(), col("dateentry").asc())
  data_lag = data_lag.select("*", lag("dateentry").over(w).alias("previousDateentry")).na.drop()
  data_lag= data_lag.select("*", lag("deviceid").over(w).alias("previousDeviceid")).na.drop()
  data_lag = data_lag.select("*", lag("latitude").over(w).alias("previousLatitude")).na.drop()
  data_lag = data_lag.select("*", lag("longitude").over(w).alias("previousLongitude")).na.drop()
  data_lag = data_lag.select("*", lag("speed").over(w).alias("previousSpeed")).na.drop()
  data_lag= data_lag.withColumn("distance",get_distance(data_lag.longitude,data_lag.latitude, data_lag.previousLongitude, data_lag.previousLatitude))
  return data_lag

#data + distance pour preparer les données 
def processing_data(data):
  #explode la colonne allnodes pour calculer le nombre de nodes 
  data_processing=data.select("*",explode(data.allnodes).alias("nodes"))
  #transformer la dateentry en date
  data_processing=data_processing.withColumn("date", to_date(from_unixtime(col('dateentry')/1000)))
  data_processing=data_processing.withColumn('token', split(data_processing['deviceid'], '#').getItem(1))
  return data_processing


#grouper par token pour ajouter les informations nécessaires 
def groupby_token(data):
  DF=data.groupby("token").agg( min("date").alias('date_premiere_cap'),
                               max("date").alias('date_derniere_cap'),
                              ((max("dateentry")-min("dateentry"))/(1000*3600) ).alias("Age_token"),
                               count("rideid").alias('nombre_trajet'),
                               count_distinct("deviceid").alias('nombre_device'),
                               sum("distance").alias('nombre_km'),
                               count("nodes").alias('nombre_nodes') ,
                              )
  return DF

#ajouter plus des informations à partir le calcule qui est deja fait avant 
def info_token(data):
  #enlever le token qui a 12 device
  data=data.filter(col("nombre_device")!=12)
  #colonne fréquence
  data=(data.withColumn("frequence",col('nombre_trajet')/col("Age_token"))
  # temps_inactivite : derniere cap current_date()
           .withColumn("temps_inactivite",datediff(current_date(),col('date_derniere_cap')))
  #age de token en jour 
           .withColumn("Age_token_jour",col("Age_token")/24)
  #age de token en mois
           .withColumn("Age_token_mois",col("Age_token_jour")/30)
  #temps d'inactivité en mois
           .withColumn("temps_inactivite_mois",col("temps_inactivite")/30))
  return data

# a partir processing_data on calcule le temps d'inactivité maximale entre deux trajets différents 
def temps_inactivite(data):
  #les differents temps_inactivite pour chaque token
  windows = Window.partitionBy("token","rideid").orderBy(col("dateentry").asc())
  data_trajet = (data.withColumn('temps_cap_debut_trajet', min("dateentry").over(windows))
                     .withColumn('temps_cap_fin_trajet', max("dateentry").over(windows))
                      .withColumn('lead_temps_cap_debut_trajet', lead('temps_cap_debut_trajet').over(windows)).dropna()
                   )
    #duree entre les trajets
  data_trajet= data_trajet.withColumn("temps_inactivite_entre_trajet",(col('temps_cap_fin_trajet')- col('lead_temps_cap_debut_trajet'))/(1000*3600))
  #temps d'innactivité maximal pour chaque token entre deux trajets différents
  data_trajet=data_trajet.groupby("token").agg(max("temps_inactivite_entre_trajet").alias("temps_inactivite_max_entre_2trajets"))
  return data_trajet
  

#### Appel de fonction

In [0]:
#tous les appels
#preparer le table token
data_distance=ajouter_distance(data)
data_process=processing_data(data_distance)
data_token=groupby_token(data_process)
data_inf_token=info_token(data_token)
#calcule le d'inactivité maximal entre deux trajet pour chaque token
data_temps_inactivite=temps_inactivite(data_process)


#### Jointure

In [0]:
#Jointure pour ajouter le temps d'innactivité maximal que a fait chaque token entre deux trajets 
data_final = (data_inf_token.join(data_temps_inactivite, on= data_inf_token.token == data_temps_inactivite.token,how = "left")
                            .drop(data_temps_inactivite.token))

####Enregistrement des données processés pour le jour précédent

In [0]:
data_final.write.format("delta").mode("append").saveAsTable("token_db.info_token")




In [0]:
#jointure apres stocke dans le table
